# Transposing data of genre, actors, directors and writers
### Why? - To simplify the work in Tableau

In [42]:
# import all packages
import pandas as pd
import numpy as np
import psycopg2 as psycopg2
import sql_functions as sqlf
import string as string

In [43]:
# pd.set_option("display.max.columns", 264)

In [44]:
schema = "capstone_24_4_group1"

In [45]:
imdb_query = f'''   SELECT *
                    FROM {schema}."IMDB_tickets_data"
                    '''

In [46]:
imdb_tick_df = sqlf.get_dataframe(imdb_query)

In [47]:
imdb_tick_df.head(5)

,tconst,primary_title,original_title,year,runtime,num_votes,average_rating,genres_count,genre,genre2,...,writers_count,writer_name,writer2_name,writer3_name,country,EU_since_1996,EU_tickets_sold,distributor,NA_gross_sales,NA_tickets_sold
0,tt0035423,Kate & Leopold,Kate & Leopold,2001,118.0,89944,6.4,3.0,Comedy,Fantasy,...,2.0,Steven Rogers,James Mangold,None,US,2497656.0,2481644.0,Miramax,47095453.0,8245453.0
1,tt0117786,Mr. Nice Guy,Yat goh ho yan,1997,88.0,29154,6.2,3.0,Action,Adventure,...,2.0,Fibe Ma,Edward Tang,None,None,NaN,NaN,New Line,12716953.0,2711503.0
2,tt0118301,Dead Man on Campus,Dead Man on Campus,1998,96.0,16474,6.0,1.0,Comedy,None,...,4.0,Anthony Abrams,Adam Larson Broder,None,None,NaN,NaN,Paramount Pictures,15064948.0,3212142.0
3,tt0118564,Affliction,Affliction,1997,114.0,19572,6.9,3.0,Drama,Mystery,...,2.0,Russell Banks,Paul Schrader,None,None,NaN,NaN,Lionsgate,6238175.0,1227987.0
4,tt0118589,Glitter,Glitter,2001,104.0,24170,2.4,3.0,Drama,Music,...,2.0,Cheryl L. West,Kate Lanier,None,None,NaN,NaN,20th Century Fox,4273372.0,755012.0


## Extract Columns

In [48]:
ww_df = imdb_tick_df[["tconst","country"]]

In [49]:
ww_df.head(5)

,tconst,country
0,tt0035423,US
1,tt0117786,None
2,tt0118301,None
3,tt0118564,None
4,tt0118589,None


In [50]:
columns_df = ww_df["country"].str.split(",", expand=True)

columns_df.columns = [f'Spalte{i+1}' for i in range(columns_df.shape[1])]
columns_df.fillna(np.nan, inplace=True)

ww_df = pd.concat([ww_df, columns_df], axis =1)

ww_df = ww_df.drop(columns="country")

ww_df

,tconst,Spalte1,Spalte2,Spalte3,Spalte4,Spalte5,Spalte6,Spalte7
0,tt0035423,US,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0117786,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0118301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0118564,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0118589,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6336,tt9883832,US,DE,NaN,NaN,NaN,NaN,NaN
6337,tt9883996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6338,tt9907782,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6339,tt9908390,FR,BE,NaN,NaN,NaN,NaN,NaN


In [51]:
imdb_countries = ww_df[["tconst", "Spalte1", "Spalte2", "Spalte3", "Spalte4", "Spalte5", "Spalte6", "Spalte7"]]

df_melted_countries = pd.melt(imdb_countries, id_vars=['tconst'], value_vars=["Spalte1", "Spalte2", "Spalte3", "Spalte4", "Spalte5", "Spalte6", "Spalte7"],
                    var_name='Spalte', value_name='country')

df_melted_countries = df_melted_countries[["tconst","country"]]
df_melted_countries.head(3)

df_countries = df_melted_countries

df_countries

,tconst,country
0,tt0035423,US
1,tt0117786,NaN
2,tt0118301,NaN
3,tt0118564,NaN
4,tt0118589,NaN
...,...,...
44382,tt9883832,NaN
44383,tt9883996,NaN
44384,tt9907782,NaN
44385,tt9908390,NaN


In [52]:
#Check
df_countries[df_countries["tconst"] == "tt9883832"]

,tconst,country
6336,tt9883832,US
12677,tt9883832,DE
19018,tt9883832,NaN
25359,tt9883832,NaN
31700,tt9883832,NaN
38041,tt9883832,NaN
44382,tt9883832,NaN


In [53]:
# Drop NA
df_countries.dropna(subset=["country"], inplace=True)

In [54]:
#Check
df_countries[df_countries["tconst"] == "tt9883832"]

,tconst,country
6336,tt9883832,US
12677,tt9883832,DE


In [55]:
df_countries["country"].unique()

array(['US', 'IT', 'HK', 'FR', 'AU', 'SE', 'RU', 'GBinc', 'GB', 'DE',
       'ES', 'CA', 'NL', 'PL', 'BR', 'TR', 'AT', 'DK', 'NO', 'CZ', 'JP',
       'TW', 'BE', 'AR', 'GR', 'MX', 'IN', 'IR', 'NZ', 'CN', 'FI',
       'FRinc', 'TH', 'KR', 'ZA', 'CS', 'KZ', 'IE', 'SK', 'RO', 'RS',
       'LU', 'SA', 'ESinc', 'CH', 'IS', 'PE', 'UA', 'LB', ' FR', ' GB',
       ' DE', ' NO', ' CH', ' US', ' CA', ' NZ', ' RO', ' ES', ' JP',
       ' AU', ' IT', ' IE', ' SE', ' IN', ' BG', ' BE', ' CN', ' CZ',
       ' NL', ' DK', ' CU', ' AT', ' MX', ' SK', ' HK', ' HU', ' SG',
       ' LU', ' MN', ' ZA', ' RU', ' AR', ' TW', ' AE', ' CY', ' KR',
       ' TR', ' BR', ' RS', ' LV', ' IS', ' MC', ' SN', ' PL', ' FI',
       ' MT', ' MR', ' TN', ' YU', ' PT', ' IL', ' TH', ' DZ', ' EE',
       ' LT', ' UA', ' CL', ' GR', ' KY', ' NP', ' MA', ' JO', ' SI',
       ' CO', ' PE'], dtype=object)

### Unify countries

In [56]:
df_countries["country"] = df_countries["country"].str.replace("inc", "")
df_countries["country"].unique()
df_countries["country"] = df_countries["country"].str.strip()

### Read-In Country list for transformation to long language format

In [57]:
df_country_names = pd.read_csv(".\Data\country_list.csv", delimiter=";", encoding="latin-1")

In [58]:
df_country_names.head(10)

,short_name,long_name
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Antigua and Barbuda
4,AI,Anguilla
5,AL,Albania
6,AM,Armenia
7,AO,Angola
8,AQ,Antarctica
9,AR,Argentina


In [59]:
df_country_names[df_country_names["short_name"] == "BE"]

,short_name,long_name
19,BE,Belgium


In [60]:
df_country_names["short_name"] = df_country_names["short_name"].str.strip()

In [61]:
df_countries = df_countries.merge(df_country_names, how ="left", left_on="country", right_on="short_name")

In [62]:
df_countries.drop(columns=["country","short_name"], inplace=True)

In [63]:
# df_countries = df_countries[["tconst","long_name"]]

### Download boxoffice_inflated

In [64]:
schema = "capstone_24_4_group1"

In [65]:
imdb_query = f'''   SELECT *
                    FROM {schema}."box_office_data_inflated"
                    '''

In [66]:
bx_infl = sqlf.get_dataframe(imdb_query)

In [67]:
bx_infl.head(5)

,tconst,country,values,inflated_values
0,tt7399138,Albania,483.0,566.208970
1,tt10223460,Albania,30528.0,32280.176916
2,tt1051906,Albania,3359.0,3874.880639
3,tt10665342,Albania,9906.0,10474.562124
4,tt10954984,Albania,4186.0,4426.258535


In [68]:
bx_infl["country"].unique()

array(['Albania', 'Argentina', 'Aruba', 'Australia', 'Austria', 'Bahrain',
       'Baltic states', 'Bangladesh', 'Belgium', 'Bolivia', 'Bosnia',
       'Bosnia and herzegovina', 'Brazil', 'Bulgaria', 'Cambodia',
       'Canada', 'Central america', 'Chile', 'Central america+', 'China',
       'Colombia', 'Costa rica', 'Croatia', 'Curaçao', 'Cyprus',
       'Czech republic', 'Czech republic/slovakia', 'Denmark', 'Djibouti',
       'Domestic', 'Dominican republic', 'E/w africa', 'East africa',
       'Ecuador', 'Egypt', 'El salvador', 'Estonia', 'Ethiopia',
       'Finland', 'France', 'Germany', 'Ghana', 'Greece',
       'Greece and cyprus', 'Guatemala', 'Honduras', 'Hong kong',
       'Hungary', 'Iceland', 'India', 'Indonesia', 'Iraq', 'Ireland',
       'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya', 'Kuwait',
       'Laos', 'Latvia', 'Lebanon', 'Lebanon/united arab emirates',
       'Lesser antilles', 'Lithuania', 'Malaysia', 'Malta', 'Mauritius',
       'Mexico', 'Middle east

In [69]:
bx_countries = pd.read_csv("./Data/bx_countries.csv", sep=";")

In [70]:
bx_countries.head(2)

,bx_country,long_name
0,Albania,Albania
1,Argentina,Argentina


In [71]:
bx_infl = bx_infl.merge(bx_countries, how="left", left_on="country", right_on="bx_country")
bx_infl = bx_infl[["tconst","bx_country","values", "inflated_values"]]
bx_infl.rename(columns={"bx_country" : "country"}, inplace=True)

In [72]:
bx_infl

,tconst,country,values,inflated_values
0,tt7399138,Albania,483.0,5.662090e+02
1,tt10223460,Albania,30528.0,3.228018e+04
2,tt1051906,Albania,3359.0,3.874881e+03
3,tt10665342,Albania,9906.0,1.047456e+04
4,tt10954984,Albania,4186.0,4.426259e+03
...,...,...,...,...
153294,tt9883996,World,6572790.0,7.582250e+06
153295,tt9896876,World,331328.0,3.713260e+05
153296,tt9907782,World,4588389.0,5.142300e+06
153297,tt9908390,World,3507860.0,4.046603e+06


## Display all to check

In [73]:
display(df_countries.head(10))
display(bx_infl.head(10))

,tconst,long_name
0,tt0035423,United States of America
1,tt0118635,Italy
2,tt0118661,United States of America
3,tt0118694,Hong Kong
4,tt0118715,United States of America
5,tt0118747,United States of America
6,tt0118834,France
7,tt0118843,France
8,tt0118892,United States of America
9,tt0118929,Australia


,tconst,country,values,inflated_values
0,tt7399138,Albania,483.0,566.208970
1,tt10223460,Albania,30528.0,32280.176916
2,tt1051906,Albania,3359.0,3874.880639
3,tt10665342,Albania,9906.0,10474.562124
4,tt10954984,Albania,4186.0,4426.258535
5,tt11358390,Albania,4262.0,4262.000000
6,tt12003946,Albania,5425.0,5736.371848
7,tt12921446,Albania,6228.0,6228.000000
8,tt13223398,Albania,5775.0,6106.460354
9,tt14109724,Albania,36584.0,38683.765470


## Upload

In [74]:
schema

'capstone_24_4_group1'

In [76]:
dataframes = {
    "countries":df_countries,
    "box_office_inflated_country_correction":bx_infl
}

In [77]:
# table_name = 'eu_merged'
engine = sqlf.get_engine()
engine
# Write records stored in a dataframe to SQL database
if engine != None:
    try:
        for table_name, df in dataframes.items():
            df.to_sql(table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='replace', # Drop the table before inserting new values 
                            schema=schema, # your class schema
                            index=False, # Write DataFrame index as a column
                            chunksize=5000, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was uploaded successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print("nope") 

The countries table was imported successfully.
The box_office_inflated_country_correction table was imported successfully.
